In [1]:
import functions
import models
import pandas as pd
import random
import json

random.seed(2021)

X_train, X_val, X_test, y_train, y_val, y_test = functions.get_data_split_and_standardise()


# Setting a baseline
We will set a baseline performance using a simple model: logistic regression. We will use L2 regularisation. Training and validation scores can be seen in the table below.

In [2]:
lr_l2 = models.LogisticRegressionModel(X_train, X_val, y_train, y_val, penalty='l2')
pd.DataFrame(lr_l2.results)

,name,penalty,training score,validation score
0,Logistic Regression,l2,0.648148,0.642857


# Evaluation of different models

Performance of four models evaluated.

In [3]:
names = [
    "Nearest Neighbors", 
    "SVM", 
    "Random Forest", 
    "AdaBoost"
]
classifiers = [
    models.KNeighborsClassifier(),
    models.SVC(),
    models.RandomForestClassifier(),
    models.AdaBoostClassifier()
]
pd.DataFrame(names, columns=["Model"])

,Model
0,Nearest Neighbors
1,SVM
2,Random Forest
3,AdaBoost


In [4]:
# create a dictionary of hyperparameters to be used in a grid search
model_dict = {}

model_dict = {
    name: {
        'model': classifier,
        'params': {}
    } for name, classifier in zip(names, classifiers)}

# set the hyperparams for each model
model_dict['Nearest Neighbors']['params'] = {
    'n_neighbors': [1 ,2 ,3 ,5 ,10]
}
model_dict['SVM']['params'] = {
    'kernel': ['linear', 'rbf'],
    'C': [0.05, 0.1, 2, 40]
}
model_dict['Random Forest']['params'] = {
    'max_depth': [5, 7, 10], 
    'n_estimators': [5, 10, 15], 
    'max_features': [1, 2, 3]    
    }
model_dict['AdaBoost']['params'] = {
    'n_estimators': [20, 35, 50], 
    }

grid_search_results = functions.train_all_models(model_dict, X_train, X_val, y_train, y_val)

EVALUATING Nearest Neighbors.
EVALUATING SVM.
EVALUATING Random Forest.
EVALUATING AdaBoost.


# Results of the grid search

In [18]:
results_dict = {}
df = pd.DataFrame(lr_l2.results).drop('penalty', axis=1)

for name in grid_search_results:
    my_dict = json.loads(grid_search_results[name])
    results_dict[name] = my_dict

results_dict
df2 = pd.DataFrame(results_dict).transpose()
pd.concat([df, df2]).set_index('name')

,training score,validation score,n_neighbors,kernel,C,max_depth,n_estimators,max_features
name,,,,,,,,
Logistic Regression,0.648148,0.642857,NaN,NaN,NaN,NaN,NaN,NaN
KNN,0.703704,0.642857,3,NaN,NaN,NaN,NaN,NaN
SVM,0.62963,0.714286,NaN,linear,40.0,NaN,NaN,NaN
RandomForest,0.814815,0.642857,NaN,NaN,NaN,5,5,1
AdaBoost,0.648148,0.571429,NaN,NaN,NaN,NaN,20,NaN
